# 🧪 SP4 Data Wrangling  PII

**Objetivo del webinar:**  
Desarrollar habilidades prácticas para la exploración, transformación y visualización de datos usando Pandas, dentro del proceso de análisis de datos en Python.

## 🎯 Objetivos principales de la sesión

1. **Identificar y manipular tipos de datos** en un DataFrame para asegurar una base sólida en el preprocesamiento.
2. **Aplicar transformaciones y crear nuevas variables** para enriquecer la información disponible y facilitar el análisis.
3. **Construir visualizaciones básicas** para explorar relaciones y comunicar hallazgos de forma efectiva.


## 🩺  Datos de salud pública

Imagina que formas parte del equipo de análisis de una institución de salud pública. Se te ha asignado la tarea de explorar y analizar un conjunto de datos con información de pacientes atendidos por una enfermedad respiratoria durante el **2023**. El objetivo es identificar patrones relevantes, agrupar información clave y generar visualizaciones que apoyen la toma de decisiones clínicas y epidemiológicas.

El conjunto de datos incluye variables como: edad, sexo, fechas de ingreso y egreso hospitalario, región geográfica, síntomas reportados, diagnóstico confirmado, resultado de laboratorio, días de hospitalización y estado final del paciente.

### 🎯 Objetivos del caso práctico

1. **Limpiar y preparar los datos** para asegurar su calidad y consistencia antes del análisis.
2. **Generar nuevas variables derivadas**, como duración de hospitalización y grupos de edad, para facilitar la segmentación y el análisis.
3. **Explorar visualmente los datos** mediante histogramas, gráficos de barras y líneas, para comprender mejor la distribución de los casos y su evolución en el tiempo.


### Diccionario de datos

| Columna             | Descripción                                                                 |
|---------------------|------------------------------------------------------------------------------|
| id_paciente         | Identificador único asignado a cada paciente.                               |
| edad                | Edad del paciente en años.                                                  |
| sexo                | Sexo del paciente, puede ser "masculino" (M) o "femenino" (F).              |
| fecha_ingreso       | Fecha en la que el paciente ingresó al hospital.                            |
| fecha_egreso        | Fecha en la que el paciente egresó del hospital.                            |
| region              | Región geográfica donde reside el paciente.                                 |
| sintoma_principal   | Síntomas que presentó el paciente.                                          |
| diagnostico         | Diagnóstico médico confirmado para el paciente.                             |
| resultado_laboratorio | Resultados de las pruebas de laboratorio realizadas al paciente.         |
| estado_paciente     | Estado final/actual del paciente.                                           |
| spo2                | Nivel de saturación de oxígeno en la sangre, expresado como porcentaje.     |
| presion_sanguinea   | Medida de la presión arterial del paciente, expresada como "sistólica/diastólica". |


### Cargado de datos y herramientas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Datos en google sheets por si desean replicar en casa
# https://docs.google.com/spreadsheets/d/1KXcXTFe02SqSKsmd6S36bbbwVkVwZNUFZq-aYsDX-i4/edit?usp=sharing
PATH="/content/base_de_datos_de_salud_publica - base_de_datos_de_salud_publica.csv" ## Agrega aquí la ruta de tu archivo

In [ ]:
datos = pd.read_csv(PATH)
datos

### Familizarización con los datos

In [ ]:
datos.info()

> ¿Los datos tiene el formato adecuado? 🤔


### Conversión de datos

#### Numéricos

In [ ]:
# Casteo de string a int
datos['edad'].astype(int) # Conversion no segura

In [ ]:
datos['edad'] = pd.to_numeric(datos['edad'], errors='coerce') # Conversión segura

> `pd.to_numeric()` es más seguro que `.astype(int)` porque convierte errores en `NaN` en lugar de fallar.  
> Útil cuando no estás seguro de que todo sea número.

In [ ]:
datos['presion_arterial']

In [ ]:
# Divide en dos columnas con el método str.split(expand=True)
presion_split = datos['presion_arterial'].str.replace('mmHg', '').str.strip().str.split("/", expand=True)

# Asigna los valores a nuevas columnas
datos['presion_arterial_sistolica'] = pd.to_numeric(presion_split[0], errors='coerce')
datos['presion_arterial_distolica'] = pd.to_numeric(presion_split[1], errors='coerce')

datos

#### Fechas

---

> **DATETIME**: Objeto que representa fechas y horas en pandas, permitiendo operaciones como filtrado, extracción y conversión de componentes temporales.


---

In [ ]:
pd.to_datetime(datos['fecha_ingreso']).info()

In [ ]:
datos['fecha_ingreso'] = pd.to_datetime(datos['fecha_ingreso'],format='%Y-%m-%d', errors='coerce')
datos['fecha_egreso'] = pd.to_datetime(datos['fecha_egreso'],format='%Y-%m-%d', errors='coerce')

---


##### Parámetros `format` y `errors` en `pd.to_datetime`

- **`format`**: Define el formato exacto de la fecha para mejorar rendimiento y evitar ambigüedades.  
  Se usa con códigos de formato tipo `strftime`, por ejemplo:
  - `%Y-%m-%d` → `2023-12-31` (ISO estándar)
  - `%d/%m/%Y` → `31/12/2023`
  - `%m-%d-%Y %H:%M` → `12-31-2023 15:45`
  - `%Y-%m-%dT%H:%M:%SZ` → con zona horaria en UTC (ISO 8601 con Z)

- **`errors`**: Maneja errores durante la conversión.
  - `"raise"` (default): lanza error si hay un valor inválido.
  - `"coerce"`: convierte errores en `NaT` (Not a Time).
  - `"ignore"`: devuelve el original sin convertir.

##### Formatos que pandas detecta automáticamente
Pandas reconoce muchas estructuras comunes sin necesidad de especificar `format`, como:
- `2023-12-31`
- `31/12/2023`
- `Dec 31, 2023`
- `2023-12-31T23:59:59Z` (formato ISO con zona horaria)
- `2023-12-31 23:59:59+00:00` (con offset de zona horaria)

##### Referencia para profundizar en `format` de fechas

Puedes revisar la documentación oficial de Python sobre `strftime` y `strptime` aquí:  
[https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)

Ahí encontrarás todos los códigos disponibles como `%Y` (año), `%m` (mes), `%d` (día), `%H` (hora), entre otros.



---

In [ ]:
datos.info()

In [ ]:
datos.head()

### Ingeniería de características

In [ ]:
#Crea una estimación del año de nacimiento del paciente
datos['año_nacimiento']= 2023 - datos['edad']

In [ ]:
datos.head()

In [ ]:
#Determina los dias que el paciente estuvo hospitalizado
datos['dias_hospitalizacion']= datos['fecha_egreso']-datos['fecha_ingreso']

In [ ]:
datos.head()

In [ ]:
datos['fecha_ingreso'].dt.year.unique()

In [ ]:
# Convierte los dias a numeros en vez de ventana de tiempo
datos['dias_hospitalizacion'] = datos['dias_hospitalizacion'].dt.days

In [ ]:
datos.info()

Si tenemos una columna de dia, otra de mes y otra se año, cómo podríamos construir una sola columna de fecha?

In [ ]:
data = {'day': [15, 28, 5, 10],
        'month': [1, 2, 3, 4],
        'year': [2023, 2024, 2025, 2023]}
df = pd.DataFrame(data)

#Construir la columna de fecha
df['Fecha'] = pd.to_datetime(df[['year', 'month', 'day']])

---

`apply()` permite aplicar funciones a filas o columnas. Usa `axis=0` para columnas y `axis=1` para filas, permitiendo operaciones personalizadas sobre cada elemento.

---

In [ ]:
#Crear columna “grupo de edad” (niño, adulto, adulto mayor)
def edad_a_grupo(edad):
    if edad < 18:
        return 'Niño'
    elif edad < 65:
        return 'Adulto'
    else:
        return 'Adulto mayor'

datos['grupo_edad'] = datos['edad'].apply(edad_a_grupo)
datos


In [ ]:
#Generar una bandera de “caso grave” si la persona estuvo más de 7 días hospitalizada o falleció

def caso_grave(row):
  dias_hospitalizacion = row['dias_hospitalizacion']
  estado_paciente = row['estado_paciente']
  return dias_hospitalizacion > 7 or estado_paciente.lower() == 'fallecido'


datos['caso_grave'] = datos.apply(caso_grave, axis=1)
datos

### Transformación de datos

#### Agregación de variables`Agg`

El equipo de salud pública necesita comprender cómo varían la edad, la duración de la hospitalización y la gravedad de la enfermedad entre los diferentes grupos de edad. Esto les permitirá identificar grupos de mayor riesgo y adaptar las estrategias de prevención y tratamiento. Por ejemplo, si se observa que los adultos mayores tienen una mayor duración de hospitalización y una mayor cantidad de casos graves, se podrían implementar medidas específicas para proteger a este grupo vulnerable.

---

`.agg()` es una función de Pandas que permite aplicar una o varias funciones de agregación a un DataFrame o Series, usualmente después de agrupar con `groupby()`.

**Parámetros:**

* **`func`:** Puede ser:
    * Una función:  `np.mean`, `sum`, `min`, `max`, etc.
    * Una lista de funciones:  `[np.mean, 'sum']`
    * Un diccionario: `{columna: función, otra_columna: ['min', 'max']}`
    * listado nombres: `nombre_1= (columna_1, función_1), nombre_2= (columna_2, función_2), nombre_3= (columna_3, función_3)`

**Funciones Default:**

Algunas funciones comunes accesibles directamente son:
* `'mean'`
* `'sum'`
* `'size'` (cuenta elementos)
* `'count'` (cuenta valores no nulos)
* `'std'` (desviación estándar)
* `'var'` (varianza)
* `'min'`
* `'max'`
* `'first'` (primer valor)
* `'last'` (último valor)
* `'nunique'` (cuenta valores únicos)


---

In [ ]:
# Muestra una tabla que indique la duración promedio de la hospitalización  y la cantidad de casos graves por grupo de edad

(datos
 .groupby('grupo_edad',as_index=False)
 .agg(
     dias_hospitalizacion_avg=('dias_hospitalizacion', 'mean'),
     total_de_casos = ('id_paciente','count'),
     casos_graves_count=('caso_grave', 'sum'),
     casos_graves_rate = ('caso_grave', 'mean')
    )
  .sort_values('casos_graves_rate', ascending=False)
)

>¿Qué sugieren los datos?

#### Tablas dinámicas `pivot_tables`

Se requiere un análisis de la distribución geográfica de los casos y su relación con el estado del paciente. Se busca identificar si existen regiones con mayor incidencia de casos o con mayor mortalidad. Esto podría ayudar a comprender factores geográficos que influyen en la propagación de la enfermedad y a asignar recursos de manera eficiente a las zonas más afectadas.

---

`pivot_table` es una función de Pandas que crea una tabla dinámica a partir de un DataFrame. Permite resumir y agregar datos de forma flexible, similar a las tablas dinámicas en hojas de cálculo.

**Parámetros:**

* **`data`:** El DataFrame a utilizar.
* **`values`:** La columna o columnas a agregar (opcional, si se omite se usa `aggfunc='size'` para contar).
* **`index`:** La columna o columnas a usar como índice (filas de la tabla).
* **`columns`:** La columna o columnas a usar como columnas de la tabla.
* **`aggfunc`:** La función o funciones de agregación a aplicar (por defecto `'mean'`).
    * Puede ser una función, una lista de funciones, o un diccionario que mapea columnas a funciones.
    * `'size'` cuenta la cantidad de elementos en cada grupo.





---

In [ ]:
(
datos
  .groupby(['region','estado_paciente'],as_index=False)
  .agg(casos=('id_paciente','count'))
)

In [ ]:
#Crea una pivot_table que muestre la cantidad de pacientes por región geográfica
# y estado del paciente (recuperado, fallecido).
datos.pivot_table(
    index='region',
    columns='estado_paciente',
    values='id_paciente',
    aggfunc='count'
)

#### Unir dataframes

Para comprender mejor la situación, necesitan comparar la tasa de casos graves de cada región con la tasa nacional. Esto les permitirá identificar las regiones con tasas significativamente más altas o más bajas que el promedio nacional, lo que podría indicar la necesidad de intervenciones específicas o la asignación de recursos adicionales.

---


`merge()` es una función de Pandas que permite combinar dos DataFrames en función de una o más columnas clave. Es similar a las operaciones JOIN en bases de datos relacionales.

**Funcionamiento:**

`merge()` busca coincidencias entre las columnas clave especificadas en ambos DataFrames y crea un nuevo DataFrame que contiene las columnas de ambos DataFrames, combinadas en función de las coincidencias.

**Parámetros clave:**

* **`left`:** El primer DataFrame.
* **`right`:** El segundo DataFrame.
* **`on`:** Nombre de la columna o columnas clave comunes en ambos DataFrames.
* **`how`:** Tipo de unión a realizar:
    * `'left'` (por defecto): Unión izquierda.
    * `'right'` : Unión derecha.
    * `'inner'` : Unión interna.
    * `'outer'` : Unión externa.



---

In [ ]:
casos_graves_rates=(datos
                    .groupby('region',as_index=False)
                    .agg(casos_graves_rate = ('caso_grave', 'mean'))
                    )

casos_graves_rates['casos_graves_rate'] = casos_graves_rates['casos_graves_rate'].round(2)
casos_graves_rates

In [ ]:
referencia_nacional=[
    {'region': 'Centro', 'Tasa Nacional': 0.35},
    {'region': 'Este', 'Tasa Nacional': 0.54},
    {'region': 'Norte', 'Tasa Nacional': 0.45},
    {'region': 'Oeste', 'Tasa Nacional': 0.49},
    {'region': 'Sur', 'Tasa Nacional': 0.47}
]

referencia_nacional = pd.DataFrame(referencia_nacional)
referencia_nacional

In [ ]:
(
    casos_graves_rates
    .merge(referencia_nacional, on='region', how='left')
)

> ¿Qué sugieren los datos?

### Visualización de datos

El equipo de salud pública necesita comprender la distribución de la edad de los pacientes que han sido atendidos por una enfermedad respiratoria. Esto les permitirá identificar los grupos de edad más afectados y adaptar las estrategias de prevención y tratamiento de manera específica. Por ejemplo, si se observa una alta concentración de pacientes en un rango de edad particular, se podrían enfocar las campañas de vacunación o las medidas de prevención en ese grupo.

---



`.plot()` es una función para crear gráficos como líneas, dispersión, histogramas y barras.

**Uso:**
* **`kind`:** Tipo de gráfico (`'line'`, `'scatter'`, `'hist'`, `'bar'`, etc.). Por defecto: `'line'`.
* **`kwargs`:** Opciones adicionales (título, etiquetas, color, tamaño, etc.).

**Tipos de gráficos:**

* `'line'` (por defecto): Gráfico de líneas.
* `'scatter'`: Gráfico de dispersión.
* `'hist'`: Histograma.
* `'bar'`: Gráfico de barras.
* `'box'`: Diagrama de caja.
* `'pie'`: Gráfico circular.
* `'kde'`: Gráfico de densidad.



**Referencias**

* [Dataframe plot](https://pandas.pydata.org/pandas-docs/version/2.1.3/reference/api/pandas.DataFrame.plot.html)


---

## ¿Cómo varia la presion arterial de los pacientes?

In [ ]:
datos['presion_arterial_sistolica'].plot(kind='hist', bins=10, edgecolor='black',color='green')
plt.title('Distribución de la presion sistólica de los pacientes')
plt.xlabel('Presion (mmHg)')
plt.ylabel('Frecuencia')

In [ ]:
datos['presion_arterial_distolica'].plot(kind='hist', bins=25, edgecolor='black')
plt.title('Distribución de la presion distólica de los pacientes')
plt.xlabel('Presion (mmHg)')
plt.ylabel('Frecuencia')

In [ ]:
datos['presion_arterial_distolica'].min()

## ¿Cómo se distribuyen por grupo de edad?

In [ ]:
valores_por_grupo_de_edad=datos['grupo_edad'].value_counts()
valores_por_grupo_de_edad.plot(kind='bar', edgecolor='black', rot=0)
plt.title('Distribución de la edad de los pacientes')
plt.xlabel('Grupo de edad')
plt.ylabel('Casos')

In [ ]:
valores_por_grupo_de_edad.plot(kind='pie', autopct='%1.2f%%')
plt.title('Distribución de la edad de los pacientes')
plt.xlabel('Grupo de edad')
plt.ylabel('Casos')